# Python для анализа данных

## Бот в Telegram

### Сбор статистики по Коронавирусу

*Ян Пиле, Яндекс.Маркет*

### Есть сайт с информацией о распространении вируса в мире
Я предлагаю достать оттуда информацию в виде Pandas dataframe а затем по запросу пользователя в телеграм выгружать ее

Импортируем requests и запишем адрес сайта со статистикой по COVID в переменную

In [1]:
import requests
url1 = 'https://www.worldometers.info/coronavirus/'

После чего через get-запрос достанем текст страницы со статистикой и распарсим ее с помощью BeautifulSoup

In [12]:
url = 'https://www.worldometers.info/coronavirus/'
website = requests.get(url).text
soup = BeautifulSoup(website, 'lxml')
# soup

На странице две таблицы: одна отвечает за информацию на сегодняшний день (под номером 0), а вторая - за информацию на вчерашний день. Достанем обе таблицы (они лежат между тегами \<\table\>\)

In [13]:
table = soup.find_all('table')[0]
# table

Разобьем таблицу на строки (они лежат между тегами tr)

In [14]:
rows = table.find_all('tr')
# rows

Сначала достанем заголовки (между тегами th). Для примера достанем заголовок первой колонки.

In [16]:
rows[0].find_all('th')[1].get_text().strip()

'Country,Other'

Теперь мы можем достать содержимое ячеек таблицы (между тегами td). Для примера достанем первую запись первого по счету столбца

In [20]:
rows[1].find_all('td')[1].get_text().strip()

'North America'

Протестируем сборку на примере одной колонки, а потом пробежимся по их списку в цикле

In [22]:
col1 = []

col1.append(rows[0].find_all('th')[1].get_text().strip()) # отдельно добавляем заголовок

for row in rows[1:]: # начинаем со второго ряда таблицы, потому что 0 уже обработали выше
    r = row.find_all('td') # находим все теги td для строки таблицы
    col1.append(r[1].get_text().strip()) # сохраняем данные в наш список

print(col1)

['Country,Other', 'North America', 'South America', 'Europe', 'Asia', 'Africa', 'Oceania', '', 'World', 'USA', 'Brazil', 'Russia', 'India', 'UK', 'Spain', 'Peru', 'Chile', 'Italy', 'Iran', 'Mexico', 'Pakistan', 'Turkey', 'Germany', 'Saudi Arabia', 'France', 'Bangladesh', 'South Africa', 'Canada', 'Qatar', 'Colombia', 'Sweden', 'Egypt', 'Argentina', 'Belarus', 'Belgium', 'Indonesia', 'Ecuador', 'Netherlands', 'Iraq', 'UAE', 'Kuwait', 'Ukraine', 'Singapore', 'Portugal', 'Oman', 'Philippines', 'Poland', 'Panama', 'Bolivia', 'Dominican Republic', 'Switzerland', 'Afghanistan', 'Romania', 'Bahrain', 'Armenia', 'Ireland', 'Nigeria', 'Israel', 'Kazakhstan', 'Honduras', 'Japan', 'Austria', 'Ghana', 'Azerbaijan', 'Guatemala', 'Moldova', 'Serbia', 'Algeria', 'Nepal', 'S. Korea', 'Denmark', 'Cameroon', 'Morocco', 'Czechia', 'Sudan', 'Ivory Coast', 'Norway', 'Malaysia', 'Uzbekistan', 'Australia', 'Finland', 'DRC', 'Senegal', 'El Salvador', 'Kenya', 'North Macedonia', 'Haiti', 'Tajikistan', 'Ethiopi

Чтобы еще раз проверить, запустим для другой колонки - TotalCases

In [23]:
col2 = []

col2.append(rows[0].find_all('th')[2].get_text().strip()) # отдельно добавляем заголовок

for row in rows[1:]: # начинаем со второго ряда таблицы, потому что 0 уже обработали выше
    r = row.find_all('td') # находим все теги td для строки таблицы
    col2.append(r[2].get_text().strip()) # сохраняем данные в наш список

print(col2)

['TotalCases', '3,139,114', '2,189,877', '2,439,033', '2,287,856', '397,484', '9,476', '721', '10,463,561', '2,691,229', '1,373,006', '647,849', '574,926', '311,965', '296,050', '282,365', '275,999', '240,436', '227,662', '220,657', '209,337', '198,613', '195,399', '190,823', '164,260', '145,483', '144,264', '103,918', '96,088', '95,043', '67,667', '66,754', '62,268', '62,118', '61,427', '56,385', '55,665', '50,273', '49,109', '48,667', '46,195', '44,334', '43,907', '42,141', '40,070', '37,514', '34,393', '32,785', '32,125', '31,816', '31,714', '31,517', '26,970', '26,239', '25,542', '25,462', '25,133', '24,688', '21,819', '18,818', '18,476', '17,766', '17,741', '17,524', '17,409', '16,613', '14,564', '13,571', '13,564', '12,800', '12,768', '12,592', '12,385', '11,809', '9,257', '9,214', '8,866', '8,639', '8,385', '7,836', '7,214', '7,039', '6,793', '6,438', '6,366', '6,334', '5,933', '5,900', '5,846', '5,530', '5,394', '5,351', '5,296', '4,831', '4,682', '4,453', '4,256', '4,237', '4,

А теперь пройдемся циклом по 9 колонкам, сгружая данные в список списков

In [24]:
field_list = []
for i in range(9):
    col = []
    col.append(rows[0].find_all('th')[i+1].get_text().strip()) # отдельно добавляем заголовок
    for row in rows[1:]: # начинаем со второго ряда таблицы, потому что 0 уже обработали выше
        r = row.find_all('td') # находим все теги td для строки таблицы
        col.append(r[i+1].get_text().strip()) # сохраняем данные в наш список
    field_list.append(col)

А вот заголовок второго столбца

In [25]:
field_list[1][0]

'TotalCases'

Остается превратить эту структуру в DataFrsme. Для этого сначала превратим ее в словарь из списков, где ключами будут названия колонок.

In [27]:
d = dict()
for i in range(9):
    d[field_list[i][0]] = field_list[i][1:]
# d

А теперь сгружаем этот словарь в функцию pd.DataFrame и он превращается в искомый DataFrame

In [28]:
import pandas as pd 
df = pd.DataFrame(d)
df.head(10)

,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical"
0,North America,"3,139,114","+14,796","167,766",+663,"1,370,398","+10,185","1,600,950","18,913"
1,South America,"2,189,877","+3,349","83,695",+81,"1,268,518","+1,457","837,664","12,782"
2,Europe,"2,439,033","+9,799","191,289",+263,"1,369,463","+12,518","878,281","5,719"
3,Asia,"2,287,856","+31,472","56,504",+774,"1,500,747","+24,551","730,605","19,345"
4,Africa,"397,484","+1,170","9,930",+16,"190,617",+895,"196,937",920
5,Oceania,"9,476",+69,126,,"8,599",,751,1
6,,721,,15,,651,,55,4
7,World,"10,463,561","+60,655","509,325","+1,797","5,708,993","+49,606","4,245,243","57,684"
8,USA,"2,691,229","+9,418","128,936",+153,"1,122,655","+5,478","1,439,638","15,864"
9,Brazil,"1,373,006","+2,518","58,406",+21,"757,462",,"557,138","8,318"


In [29]:
df.head(8)

,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical"
0,North America,"3,139,114","+14,796","167,766",+663,"1,370,398","+10,185","1,600,950","18,913"
1,South America,"2,189,877","+3,349","83,695",+81,"1,268,518","+1,457","837,664","12,782"
2,Europe,"2,439,033","+9,799","191,289",+263,"1,369,463","+12,518","878,281","5,719"
3,Asia,"2,287,856","+31,472","56,504",+774,"1,500,747","+24,551","730,605","19,345"
4,Africa,"397,484","+1,170","9,930",+16,"190,617",+895,"196,937",920
5,Oceania,"9,476",+69,126,,"8,599",,751,1
6,,721,,15,,651,,55,4
7,World,"10,463,561","+60,655","509,325","+1,797","5,708,993","+49,606","4,245,243","57,684"


Для удобства переименуем колонки с составными названиями.

In [30]:
df = df.rename(columns={'Country,Other': 'Country', 'Serious,Critical': 'SeriousCritical'})

А теперь сгрузим весь процесс, который мы проделали, в одну функцию. Ее аргумент tag = 0 - информация за сегодняшний день, a tag=1 - информация за вчерашний день.

In [31]:
def stat(tag = 0):
    url = 'https://www.worldometers.info/coronavirus/'
    website = requests.get(url).text
    soup = BeautifulSoup(website, 'lxml')
    table = soup.find_all('table')[tag]
    rows = table.find_all('tr')
    fields_list = []

    for i in range(9):
        col = []
        col.append(rows[0].find_all('th')[i+1].get_text().strip())
        for row in rows[1:224]:
            r = row.find_all('td')
            col.append(r[i+1].get_text().strip())
        fields_list.append(col)
    d = dict()
    for i in range(9):
        d[fields_list[i][0]] = fields_list[i][1:]
    df = pd.DataFrame(d)
    df = df.rename(columns = {'Country,Other':'Country', 'Serious,Critical':'SeriousCritical'})
    return df

В первых 8 строках таблицы в колонке Country лежат регионы + Мир. Одна строка пустая. Уберем ее с помощью спискового включения.

In [32]:
print(*list(i+'\n' for i in list(stat()['Country'][:8]) if i!=''))

North America
 South America
 Europe
 Asia
 Africa
 Oceania
 World



А с 8 по 220 строки лежат названия стран. Выведем ее с помощью спискового включения по 6 стран в линию (для этого вставим символ переноса строки)

In [33]:
x = stat()['Country'][8:220]
import re
# print(x)
print(', '.join([e+'\n' if i%6 == 5 else e for i,e in enumerate(x)]).replace('\n,',',\n'))


USA, Brazil, Russia, India, UK, Spain,
 Peru, Chile, Italy, Iran, Mexico, Pakistan,
 Turkey, Germany, Saudi Arabia, France, Bangladesh, South Africa,
 Canada, Qatar, Colombia, Sweden, Egypt, Argentina,
 Belarus, Belgium, Indonesia, Ecuador, Netherlands, Iraq,
 UAE, Kuwait, Ukraine, Singapore, Portugal, Oman,
 Philippines, Poland, Panama, Bolivia, Dominican Republic, Switzerland,
 Afghanistan, Romania, Bahrain, Armenia, Ireland, Nigeria,
 Israel, Kazakhstan, Honduras, Japan, Austria, Ghana,
 Azerbaijan, Guatemala, Moldova, Serbia, Algeria, Nepal,
 S. Korea, Denmark, Cameroon, Morocco, Czechia, Sudan,
 Ivory Coast, Norway, Malaysia, Uzbekistan, Australia, Finland,
 DRC, Senegal, El Salvador, Kenya, North Macedonia, Haiti,
 Tajikistan, Ethiopia, Venezuela, Gabon, Guinea, Kyrgyzstan,
 Bulgaria, Djibouti, Bosnia and Herzegovina, Luxembourg, Mauritania, Hungary,
 French Guiana, CAR, Greece, Costa Rica, Thailand, Somalia,
 Croatia, Mayotte, Albania, Palestine, Cuba, Maldives,
 Madagascar, Par

Для вывода результатов в текстовом формате воспользуемся библиотекой tabulate (в таком формате мы будем отправлять результаты в telegram'е)

In [34]:
from tabulate import tabulate

Например общее количество случаев заболевания коронавирусом в Великобритании вывести можно так:

In [35]:
x = df[['Country','TotalCases']][df['Country']=='UK'].reset_index(drop=True)
print(tabulate(x, headers = x.columns, tablefmt="grid"))

+----+-----------+--------------+
|    | Country   | TotalCases   |
+====+===========+==============+
|  0 | UK        | 311,965      |
+----+-----------+--------------+
